# ADD TITLE

#### Introduction
<br>
&nbsp; A research team at UBC wants to collect data about how people play video games. To do this, they plan to recruit participants to play Minecraft on a recorded server, which will allow them to observe player actions and interactions. They wish to target their recruitment efforts to maximize the amount of data they can collect. 
<br>
<br>
&nbsp; We utilized data collected by the reasearch team to identify which kinds of players are most likely to contribute large amounts of data so that those players can be targeted in recruiting efforts. Specifically, we aimed to determine whether or not we can estimate the total playing time a participant will contribute based on their age and previous playing experience. The research team provided 2 datasets: players.csv and sessions.csv, which we will refer to as the Players Dataset and the Sessions Dataset. The Players Dataset contains 196 observations and the following 9 variables: experience, subscribe, hashedEmail, played_hours, name, gender, age, individualID, and organizationName. The Sessions Dataset contains 1535 observations and the following 5 variables: hashedEmail, start_time, end_time, original_start_time, and original_end_time. All variables and potential issues with the data are described in the tables below. We only looked at the experience, age, and played_hours variables in the Players Dataset to address our question. 
<br>
<br>

##### **Table 1: Variable Descriptions**

| Variable Name | Variable Type | Meaning |
|---------------|---------------|---------|
|experience|character|players experience level|
|subscribe|logical|whether or not player is subscribed to email updates|
|hashedEmail|character|players encrypted email address|
|played_hours|double|total hours of play time contributed|
|name|character|players name|
|gender|character|players gender|
|age|double|players age|
|individualID|logical|assumed to be a string of numbers and/or letters assigned to player for identification|
|organizationName|logical|assumed to be name of the group that was asked to participate which player is part of|
|start_time|character|start date and time of playing session|
|end_time|character|end date and time of playing session|
|original_start_time|double|start time of playing session in unix timestamp format|
|original_end_time|double|end time of playing session in unix timestamp format|
<br>
<br>

##### **Table 2: Issues with Data**

| Data Table | Issue |
|------------|-------|
|Players Dataset|individualID and organizationName columns provide no information|
|Players Dataset|experience and gender should be factors because they are categorical|
|Players Dataset|some variables don’t contain equal amounts of each type (eg. there are more males than any other gender, etc.)|
|Sessions Dataset|multiple observations with the same hashedEmail, may affect whether/how observations are grouped later on for analysis|
|Sessions Dataset|start_time and end_time columns should be datetime, not character|
|Sessions Dataset|all observations for original_start_time and original_end_time are the same because they are in unix timestamp format and the values are too large, are not providing valuable information|
|Both|data is in 2 data frames, should be in 1|
|Both|sessions data has more observations than players data, may be an issue when merging data sets if one has more unique hashed emails than the other|

#### Methods and Results

#### Discussion

#### References